In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.array as da
import os
import sys
import logging


logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis import Analysis, AnalysisManager, GR_Confirmation, SpotDetection_Confirmation

Loads in the data from specified location

In [2]:
loc = None # r"Y:\smFISH_images\Eric_smFISH_images\20230511\GR_ICC_3hr_R1_1nM_0min_050823\GR_ICC_3hr_R1_1nM_0min_050823.h5"
log_location = r'\\munsky-nas.engr.colostate.edu\share\Users\Jack\All_Analysis'  # r'Y:\Users\Jack\All_Analysis'
am = AnalysisManager(location=loc, log_location=log_location) # change this to your mounted drive location for this folder
# This analysis is best done with the NAS mounted 

In [ ]:
# list all analysis done 
am.list_analysis_names()

In [ ]:
# can filter on name and dates
am.select_analysis('ER_Jan0925_GR')
print(am.analysis)

Does analysis/confirmation

In [5]:
# select GR conformation analysis 
GR = GR_Confirmation(am)

In [6]:
# this loads the data into memory 
GR.get_data()

In [7]:
# This is a temporary fix, I didnt do this in the correct spot in the first run
# this will be fixed in future runs
for h5 in np.unique(GR.cellprops['h5_idx']):
    GR.cellprops.loc[GR.cellprops['h5_idx'] == h5, 'fov'] = pd.Categorical(GR.cellprops.loc[GR.cellprops['h5_idx'] == h5, 'fov']).codes


In [ ]:
# make sure this has all the measurements you want
GR.cellprops.keys()

In [ ]:
# run this multiple times to see a new randomly selected cell
GR.display()

In [ ]:
GR.cellprops

In [ ]:
keys_to_plot = ['nuc_area', 'nuc_intensity_mean']
channel_maps = {'-1': ' DAPI', '-0': ' GR'}
for k in GR.cellprops.keys():
    if k in keys_to_plot or k.split('-')[0] in keys_to_plot:
        # Plot histogram for 'area'
        plt.figure(figsize=(10, 5))
        plt.hist(GR.cellprops[k], bins=30, alpha=0.5, label='Area')
        plt.ylabel('Frequency')
        for suffix, channel in channel_maps.items():
            if k.endswith(suffix):
                k = k.replace(suffix, channel)
                break
        plt.title(f'Histogram of {k}')
        plt.legend()
        plt.show()

In [ ]:
# spot counts as a function of time and dex
tp_set = sorted(set(GR.cellprops['time']))
print(tp_set)
dex_set = sorted(set(GR.cellprops['Dex_Conc']))
print(dex_set)
keys_to_plot = ['nuc_area', 'nuc_intensity_mean-0']
for k in keys_to_plot:
    fig, axs = plt.subplots(len(tp_set), len(dex_set), figsize=(15, 15))
    for suffix, channel in channel_maps.items():
        if k.endswith(suffix):
            l = k.replace(suffix, channel)
            break
        else:
            l = k
    fig.suptitle(f'{l} as a function of time and dex', fontsize=16)
    for i_d, d in enumerate(dex_set):
        data = GR.cellprops[GR.cellprops['Dex_Conc'] == d]
        for i_t, t in enumerate(tp_set):
            temp = data[data['time'] == t]
            mean_val = temp[k].mean()
            std_val = temp[k].std()
            if d == 0 and t == 0:
                for ax in axs[i_t, :]:
                    ax.hist(temp[k], bins=200, density=True)
                    ax.axvline(mean_val, color='r', linestyle='solid', linewidth=2)
                    ax.axvline(mean_val + std_val, color='g', linestyle='dashed', linewidth=1)
                    ax.axvline(mean_val - std_val, color='g', linestyle='dashed', linewidth=1)
                    ax.set_xlim([0, GR.cellprops[k].max()])
                    ax.set_xticks([])
                    ax.set_yticks([])
                    ax.grid(True, axis='x', linestyle='--', linewidth=0.5)
            else:
                axs[i_t, i_d].hist(temp[k], bins=200, density=True)
                axs[i_t, i_d].axvline(mean_val, color='r', linestyle='solid', linewidth=2)
                axs[i_t, i_d].axvline(mean_val + std_val, color='g', linestyle='dashed', linewidth=1)
                axs[i_t, i_d].axvline(mean_val - std_val, color='g', linestyle='dashed', linewidth=1)
                axs[i_t, i_d].set_xlim([0, GR.cellprops[k].max()])
                axs[i_t, i_d].set_yticks([])
                axs[i_t, i_d].grid(True, axis='x', linestyle='--', linewidth=0.5)
                if i_t != len(tp_set) - 1:
                    axs[i_t, i_d].set_xticks([])
            axs[i_t, 0].set_ylabel(f'Time: {t}')
            axs[0, i_d].set_title(f'Dex: {d}')
    plt.show()



In [13]:
GR.close()